In [1]:
notebook_name = "EvaluatePredictions"

# Update only this cell with your local parameters

predictions_folder = "ChildrensPredictionArrays"
ground_truth_file  = r"ChildrensTestArrays\segmentation-test.npy"

acceptable_margin_mm = 1.0
mm_per_pixel = 1.0

roc_thresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001,
                  0.0008, 0.0006, 0.0004, 0.0002, 0.0001,
                  0.00001, 0.000001]


In [2]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os 
import scipy.ndimage

from random import sample

from evaluation_metrics import *

from local_vars import root_folder

In [3]:
# Read data arrays

predictions_folder_fullpath = os.path.join(root_folder, predictions_folder)

predictions_file_list = [f for f in os.listdir(predictions_folder_fullpath) if f.endswith('.npy')]
num_predictions = len(predictions_file_list)

print("Found {} predictions in {}".format(num_predictions, predictions_folder_fullpath))

groundtruth_fullname = os.path.join(root_folder, ground_truth_file)

print("Reading ground truth from:  {}".format(groundtruth_fullname))

groundtruth_data = np.load(groundtruth_fullname)
num_groundtruth = groundtruth_data.shape[0]

print("\nFound {} ground truth images".format(num_groundtruth))

Found 8 predictions in d:\Data\ChildrensPredictionArrays
Reading ground truth from:  d:\Data\ChildrensTestArrays\segmentation-test.npy

Found 1126 ground truth images


In [4]:
# Compute metrics

best_thresholds = np.zeros(num_predictions)
best_tps = np.zeros(num_predictions)
best_fps = np.zeros(num_predictions)
aurocs = np.zeros(num_predictions)

for i in range(num_predictions):
    prediction_filename = predictions_file_list[i]
    print("Calculating metrics for file: {} ...".format(prediction_filename))
    
    prediction_fullname = os.path.join(predictions_folder_fullpath, prediction_filename)
    prediction_data = np.load(prediction_fullname)
    
    true_positives, false_positives = compute_roc(
        roc_thresholds, prediction_data, groundtruth_data, acceptable_margin_mm, mm_per_pixel)
    
    aurocs[i] = compute_auroc(true_positives, false_positives)
    best_threshold_index = compute_goodness(roc_thresholds, true_positives, false_positives)
    best_thresholds[i] = roc_thresholds[best_threshold_index]
    best_tps[i] = true_positives[best_threshold_index]
    best_fps[i] = false_positives[best_threshold_index]

Calculating metrics for file: q000_prediction.npy ...
Calculating metrics for file: q001_prediction.npy ...
Calculating metrics for file: q002_prediction.npy ...
Calculating metrics for file: q003_prediction.npy ...
Calculating metrics for file: q004_prediction.npy ...
Calculating metrics for file: q005_prediction.npy ...
Calculating metrics for file: q006_prediction.npy ...
Calculating metrics for file: q007_prediction.npy ...


In [8]:
# Print metrics

print("\nPrediction files")
for i in range(num_predictions):
    print(predictions_file_list[i])

print("\nAUROCs")
for i in range(num_predictions):
    print(aurocs[i])

print("\nBest thresholds")
for i in range(num_predictions):
    print(best_thresholds[i])

print("\nBest true positive rates")
for i in range(num_predictions):
    print(true_positives[i])

print("\nBest false positive rates")
for i in range(num_predictions):
    print(false_positives[i])



Prediction files
q000_prediction.npy
q001_prediction.npy
q002_prediction.npy
q003_prediction.npy
q004_prediction.npy
q005_prediction.npy
q006_prediction.npy
q007_prediction.npy

AUROCs
0.9483584510785631
0.973459018468078
0.9849062909599142
0.9825783411538758
0.9593933860490936
0.9685175158124867
0.9699881866361789
0.9648691499566852

Best thresholds
0.0004
0.0002
0.002
0.0006
0.0002
0.0006
0.0001
0.0002

Best true positive rates
0.3531049569929896
0.4753890518570079
0.5422612240839259
0.5862128971312087
0.619947297767613
0.6490329637547855
0.6771491075423856
0.7069805598369214

Best false positive rates
0.0005872794060996966
0.0012885191123075401
0.0019281233384759843
0.0025538976954483416
0.003207549584028584
0.003935305614246261
0.004796133299248595
0.005963559383634305


In [6]:
# Save notebook

from IPython.display import Javascript
script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
Javascript(script)

<IPython.core.display.Javascript object>

In [7]:
# Archive notebook with unique filenames based on timestamps in one single HTML file.

timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

notebook_fullpath = os.path.join(root_folder, "LeaveOneOutNotebooks")

if not os.path.exists(notebook_fullpath):
    os.makedirs(notebook_fullpath)
    print("Creating folder: {}".format(notebook_fullpath))
notebook_file_name = notebook_name + "_" + timestamp + ".html"
notebook_fullname = os.path.join(notebook_fullpath, notebook_file_name)

os.system("jupyter nbconvert --to html "+ notebook_name +" --output " + notebook_fullname)
print("Notebook saved to: {}".format(notebook_fullname))

Notebook saved to: d:\Data\LeaveOneOutNotebooks\EvaluatePredictions_2019-10-14_13-24-15.html
